In [4]:
from tqdm import tqdm
import os,sys
from multiprocessing import Pool
from time import sleep
import asyncio
import concurrent.futures

def sub_process(num:int):
    print(f'start inner process{num}')
    sleep(0.01)
    print(f'end inner process{num}')

def process(num:int):
    cpu_core_num=os.cpu_count()
    print(f'cpu_core_num={cpu_core_num}')
    pool = Pool(min(20, cpu_core_num if cpu_core_num else 2))
    print('Start generating conformers...')
    content_inputs = [item for item in tqdm(pool.imap(sub_process, range(num)), desc="Generating conformers", file=sys.stdout)]
    pool.close()
    print(f'end process')

loop = asyncio.get_event_loop()
with concurrent.futures.ProcessPoolExecutor() as executor:
	await loop.run_in_executor(executor, process, 10000)


cpu_core_num=3
Start generating conformers...
start inner process0start inner process1start inner process2

Generating conformers: 0it [00:00, ?it/s]
end inner process0end inner process1

end inner process2start inner process3start inner process4


start inner process5
end inner process3end inner process4

start inner process6end inner process5start inner process7


start inner process8
end inner process6end inner process7

start inner process9end inner process8start inner process10


start inner process11
end inner process9end inner process10

start inner process12end inner process11
start inner process13

start inner process14
end inner process12
end inner process13start inner process15

start inner process16end inner process14

start inner process17
end inner process15
end inner process16start inner process18

start inner process19end inner process17

start inner process20
end inner process18
end inner process19start inner process21

start inner process22
end inner process20
start i

In [1]:
import json,os
from dotenv import load_dotenv, find_dotenv
# 载入环境变量
load_dotenv(find_dotenv(), override=True)

from volcengine.ApiInfo import ApiInfo
from volcengine import Credentials
from volcengine.auth.SignerV4 import SignerV4
from volcengine.base.Service import Service
from volcengine.ServiceInfo import ServiceInfo

API_INFOS = {
    "ListResourceQueues": ApiInfo("POST", "/", {
        "Action": "ListResourceQueues",
        "Version": "2021-10-01",
    }, {}, {})
}

class Client(Service):

    def __init__(self, ak, sk):
        credentials = Credentials.Credentials(
            ak=ak,
            sk=sk,
            service="ml_platform",
            region="cn-beijing",
        )
        self.service_info = ServiceInfo(
            "open.volcengineapi.com",
            {"Accept": "application/json"},
            credentials,
            10,
            10,
            "http",
        )
        self.api_info = API_INFOS
        super().__init__(self.service_info, self.api_info)



  

In [2]:
my_ak = os.getenv('AccessKeyId','')
my_sk = os.getenv('SecretAccessKey','')
client = Client(my_ak, my_sk)
request = {"Offset": 0, "Limit": 10}
res = client.json(api="ListResourceQueues", params={}, body=json.dumps(request))

In [ ]:
config={
    "Name": "my-task-name",## id name
    "EnableTensorBoard": False,
    "CodeSource": "", 
    "CodeOriPath": "", 
    "ImageSpec": {
        "Id": "ml_platform/python:2.7_3.7",
        "Description": "XXX",
        "Version": "Latest",
        "Type": "XXX", ## the Engine of the Image
        "Url": "XXX",
        "ImageCredential": {
            "RegistryUsername": "XXX", ## 拉公网镜像用户名
            "RegistryToken": "XXX" ## 拉公网镜像密码
        }
    },
    "EntrypointPath": "XXX", ## python uniqsar config path
    "Envs": [
        {
            "Name": "XXX",
            "Value": "XXX",
            "IsPrivate": False ## 是否是私有环境变量
        }
    ],
    "Framework": "Custom", ## 允许的值：'TensorFlowPS','PyTorchDDP','BytePS','Horovod','MPI','Slurm', 'Custom'
    "ActiveDeadlineSeconds": -1,
    "DelayExitTimeSeconds": 10080,
    "ResourceGroupId": "XXX",
    "TaskRoleSpecs": [
        {
            "RoleName": "XXX",
            "RoleReplicas": 1,
            "ResourceSpecId": "XXX",
            "ResourceSpec": {
                "FlavorID": "ml.c1e.large",
                "ResourceSlice": {
                    "Count": int(2),
                    "Total": int(3)
                },
                "ZoneId": "XXX"
            }
        }
    ],
    "Storages": [ ## 待挂载的数据盘列表
        {
            "Type": "Tos", ## 存储类型，可填值：Tos / Nas / Vepfs / Dataset
            "MountPath": "XXX", ## 挂载路径
            "Bucket": "XXX",## TOS 存储桶名称，仅当 Type == Tos 时必填
            "Prefix": "XXX", ## TOS 挂载 Prefix，仅当 Type == Tos 时有效
            "NasAddr": "XXX", ## NAS 挂载点地址，仅当 Type = Nas 时有效
            "NasId": "XXX", ## NAS 实例id，仅当 Type = Nas 时有效
            "NasName": "XXX", ## NAS 实例名称，仅当 Type = Nas 时有效
            "VepfsName": "XXX", ## VEPFS 实例名称，仅当 Type = Vepfs 时有效
            "Credential": { ## 用于指定凭证（AK/SK），仅当 Type == Tos 时有效
                "AccessKeyId": "XXX",
                "SecretAccessKey": "XXX"
            },
            "ReadOnly": False, ## 是否只读挂载，仅当 Type == Tos 时有效
            "SubPath": "XXX", ## 子目录路径，仅当 Type == Vepfs 时有效
            "VepfsId": "XXX", ## VEPFS 实例 ID, 仅当 Type = Vepfs 时有效
            "FsName": "XXX", ## CloudfsFileSystem 实例名字
            "DatasetId": "XXX", ## 数据集Id，仅当 Type = Dataset 有效
            "AnnotationId": "XXX" ## 标注集Id, 仅当 Type = Annotation 有效
        }
    ],
    "TensorBoardPath": "XXX", ## 如果开启 TensorBoard，默认值 /tensorboard_logs。
    "Description": "",
    "Credential": {
        "AccessKeyId": "XXX",
        "SecretAccessKey": "XXX"
    },
    "SidecarMemoryRatio": 0.5, ## 分配给 sidecar container memory 的比例，允许的值(0,1), 例如 0.5 代表 50%分配给 sidecar 容器
    "CacheType": "Cloudfs", ## 数据缓存类型，允许的值：'Cloudfs', 'Gpfs'，默认 'Cloudfs'
    "SidecarImage": "XXX",
    "SidecarResourceCPU": 1.0,
    "SidecarResourceMemory": 1.0, ## 单位为Gi
    "ResourceQueueId": "XXX",
    "AccessTypes": [
        "XXX"
    ],
    "AccessUserIds": [
        XXX
    ],
    "Preemptible": True,
    "Priority": 4,
    "TensorBoardStorage": {
        "Type": "Tos",  ## 存储类型，可填值：Tos / Nas / Vepfs / Dataset
        "MountPath": "", ## 挂载路径
        "Bucket": "", ## TOS 存储桶名称，仅当 Type == Tos 时必填
        "Prefix": "", ## TOS 挂载 Prefix，仅当 Type == Tos 时有效
        "NasAddr": "", ## NAS 挂载点地址，仅当 Type = Nas 时有效
        "NasId": "", ## NAS 实例 Id，仅当 Type = Nas 时有效
        "NasName": "", ## NAS 实例名称，仅当 Type = Nas 时有效
        "VepfsName": "", ## VEPFS 实例名称，仅当 Type = Vepfs 时有效
        "Credential": { ## 用于指定凭证（AK/SK），仅当 Type == Tos 时有效
            "AccessKeyId": "",
            "SecretAccessKey": ""
        },
        "ReadOnly": False, ## 是否只读挂载，仅当 Type == Tos 时有效
        "SubPath": "", ## 子目录路径，仅当 Type == Vepfs 时有效
        "VepfsId": "", ## VEPFS 实例 ID, 仅当 Type = Vepfs 时有效
        "FsName": "", ## CloudfsFileSystem 实例名字
        "DatasetId": "", ## 数据集Id，仅当 Type = Dataset 有效
        "AnnotationId": "", ## 标注集Id, 仅当 Type = Annotation 有效
        "NsId": "", ## CloudfsNamespace Id
        "ExternalCFS": True, ## 标识外部传入的CloudFS请求
        "AdvancedOptions": { ## CloudFS高级选项
            "MetaCacheCapacity": "", ## CloudFS高级选项 缓存文件数量
            "MetaCacheExpiryMesc": "" ## CloudFS高级选项 缓存淘汰时间
        }
    },
    "RetryOptions": {
        "EnableRetry": True, ## 允许重试
        "MaxRetryTimes": 5, ## 最大重试次数，仅当EnableRetry = True 时有效
        "IntervalSeconds": 120, ## 重试时间间隔，仅当EnableRetry = True 时有效
        "PolicySets": [ ## 重试的策略，仅当EnableRetry = True 时有效
            "Failed"
        ]
    },
    "DiagOptions": [
        {
            "Name": "HostPing",
            "Enable": True,
            "Triggers": [
                "TaskFailed"
            ]
        }
    ],
}


In [6]:
body_data={
    "Name": "测试创建任务请求",
    "Description": "test",

    "EnableRangeType": "Range", ## 可见范围
    "EnableArr": [
        [
            "Queue" ## 队列成员可见
        ]
    ],

    "ImageSpec": {
        "Id": "ml_platform/python:2.7_3.7",
        "Type": "Preset"
    },
    
    "sourceCodeState": -1,
    "EntrypointPath": "sleep 1d",
    "ResourceQueueId": "q-20240626201235-9s8hb", ## 4090-materials: "q-20240626201235-9s8hb", 4090-public: "q-20240626201317-5rc7m", cpu-开发机: "q-20221026190811-d7npc"
    "Priority": 4,
    "Framework": "custom",
    "TaskRoleSpecs": [
        {
            "RoleName": "worker",
            "RoleReplicas": 1,
            "ResourceSpec": {
                "FlavorID": "ml.g1ie.large",
                "ResourceSlice": {
                    "Count": 0,
                    "Total": 0
                },
                "ZoneId": "cn-guilin-a"
            }
        },
        # {
        #     "RoleName": "chief",
        #     "RoleReplicas": 0,
        #     "ResourceSpec": {
        #         "FlavorID": "",
        #         "ZoneId": ""
        #     }
        # },
        # {
        #     "RoleName": "ps",
        #     "RoleReplicas": 0,
        #     "ResourceSpec": {
        #         "FlavorID": "",
        #         "ZoneId": ""
        #     },
        # },
        # {
        #     "RoleName": "evaluator",
        #     "RoleReplicas": 0,
        #     "ResourceSpecId": "",
        #     "ResourceSpec": {
        #         "FlavorID": "",
        #         "ZoneId": ""
        #     }
        # }
    ],
    "Storages": [
        {
            "MountPath": "/vepfs/fs_users",
            "Type": "Vepfs",
            "VepfsName":'vepfs',
            "SubPath":"fs_users",
            "VepfsId":"vepfs-cnbj2183960b0bac",
            "VepfsHostPath":"/vepfs/fs_users"
        }
    ],
    "ActiveDeadlineSeconds": 864000,
    "DelayExitTimeSeconds": 300,
    "AccessTypes": [
        "Queue"
    ],
    "AccessUserIds": [],
    "Tags": [],
    "CodeSource": "",
    "CodeOriPath": "",
    "LocalCodePath": "",
    "TOSCodePath": "",
    "Preemptible": True,
    "Envs": [],
    "EnableTensorBoard": False,
    "TensorBoardPath": ""
}

In [8]:
from bohrium_open_sdk import OpenSDK

In [9]:
data={"access_key":"sk-768910d50cc64a4391cb82c9c977e513", "app_key":"qsar-web-new-uuid1721369234"}

client=OpenSDK(access_key=data["access_key"],app_key=data["app_key"])
user_info = client.user.get_info()
user_info

[2024-07-26 11:56:09 - root:67 - ERROR] HTTP Error 401 for url https://openapi.dp.tech/openapi/v1/ak/user, response: {"code":401,"error":"invalid_access_key"}


AuthenticationError: HTTP Error 401 for url https://openapi.dp.tech/openapi/v1/ak/user, response: {"code":401,"error":"invalid_access_key"}